In [ ]:
ls

drive/  sample_data/


In [ ]:
cp /content/drive/MyDrive/TFM/dataset_finetuning.zip /content/.

In [ ]:
mkdir dataset_finetuning

In [ ]:
!unzip dataset_finetuning.zip -d dataset_finetuning

Archive:  dataset_finetuning.zip
   creating: dataset_finetuning/content/drive/MyDrive/TFM/dataset_finetuning/
  inflating: dataset_finetuning/content/drive/MyDrive/TFM/dataset_finetuning/PC_11085153.jpg  
  inflating: dataset_finetuning/content/drive/MyDrive/TFM/dataset_finetuning/PC_6205969.jpg  
  inflating: dataset_finetuning/content/drive/MyDrive/TFM/dataset_finetuning/PC_3220641.jpg  
  inflating: dataset_finetuning/content/drive/MyDrive/TFM/dataset_finetuning/PC_3220639.jpg  
  inflating: dataset_finetuning/content/drive/MyDrive/TFM/dataset_finetuning/PC_3220632.jpg  
  inflating: dataset_finetuning/content/drive/MyDrive/TFM/dataset_finetuning/PC_3220602.jpg  
  inflating: dataset_finetuning/content/drive/MyDrive/TFM/dataset_finetuning/PC_2947458.jpg  
  inflating: dataset_finetuning/content/drive/MyDrive/TFM/dataset_finetuning/PC_2947424.jpg  
  inflating: dataset_finetuning/content/drive/MyDrive/TFM/dataset_finetuning/PC_1738157.jpg  
  inflating: dataset_finetuning/content/dr

In [ ]:
import pandas as pd

train_csv = '/content/drive/MyDrive/TFM/dataset/train_split.csv'
val_csv = '/content/drive/MyDrive/TFM/dataset/val_split.csv'
test_csv = '/content/drive/MyDrive/TFM/dataset/test_split.csv'

df_train = pd.read_csv(train_csv)
df_val = pd.read_csv(val_csv)
df_test = pd.read_csv(test_csv)

In [ ]:
!pip install datasets
from datasets import DatasetDict, Dataset
train_ds = Dataset.from_pandas(df_train)
val_ds = Dataset.from_pandas(df_val)
test_ds = Dataset.from_pandas(df_test)

In [ ]:
print(train_ds)
print(val_ds)
print(test_ds)

Dataset({
    features: ['image_path', 'caption', 'similarity'],
    num_rows: 3776
})
Dataset({
    features: ['image_path', 'caption', 'similarity'],
    num_rows: 519
})
Dataset({
    features: ['image_path', 'caption', 'similarity'],
    num_rows: 475
})


In [ ]:
dataset_dict = DatasetDict({
    'train': train_ds,
    'valid': val_ds,
    'test': test_ds
})

In [ ]:
train_ds[0]

{'image_path': '/content/drive/MyDrive/TFM/dataset_finetuning/LY_83215.jpg',
 'caption': 'A cell where a cerebriform nucleus is absent.',
 'similarity': 1}

In [ ]:
train_ds.features

{'image_path': Value(dtype='string', id=None),
 'caption': Value(dtype='string', id=None),
 'similarity': Value(dtype='int64', id=None)}

**Preprocessing**

In [ ]:
from transformers import AutoModel, CLIPProcessor
import torch
from PIL import Image
model=AutoModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/862k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

In [ ]:
type(dataset_dict["train"])

datasets.arrow_dataset.Dataset

In [ ]:
# 3. Función de preprocesamiento
def preprocess_data(examples):
    images = [Image.open('/content/dataset_finetuning/'+ path).convert("RGB") for path in examples['image_path']]
    inputs = processor(text=examples['caption'], images=images, return_tensors="pt", padding=True, truncation=True)
    inputs['labels'] = torch.tensor(examples['similarity'], dtype=torch.float) # O float si es regresión
    # inputs['caption'] = examples['caption']
    return inputs

# 4. Aplicar preprocesamiento
processed_dataset = dataset_dict.map(preprocess_data, batched=True, remove_columns=dataset_dict['train'].column_names)


Map:   0%|          | 0/3776 [00:00<?, ? examples/s]

Map:   0%|          | 0/519 [00:00<?, ? examples/s]

Map:   0%|          | 0/475 [00:00<?, ? examples/s]

In [ ]:
from torch.utils.data import DataLoader

# Create DataLoader for training and validation sets
train_loader = DataLoader(processed_dataset["train"], batch_size=32, shuffle=True, )
val_loader = DataLoader(processed_dataset["valid"], batch_size=64, shuffle=False)

**Fine-tuning**

In [ ]:
# pick specific layers to train (note: you can add more layers to this list)
trainable_layers_list = ['projection']

# Apply freezing configuration
for name, param in model.named_parameters():

    # freeze all params
    param.requires_grad = False

    # unfreeze layers in trainable_layers_list
    if any(layer in name for layer in trainable_layers_list):
        param.requires_grad = True

In [ ]:
import torch
import torch.nn.functional as F

def manual_contrastive_loss(embedding1, embedding2, label, margin=0.5):
    """
    Calcula la pérdida contrastiva manualmente, basada en etiquetas 0/1.

    Args:
        embedding1: Tensor de embeddings normalizados (ej. imagen) (N, embedding_dim).
        embedding2: Tensor de embeddings normalizados (ej. texto) (N, embedding_dim).
        label: Tensor de etiquetas (0 o 1) para cada par (N).
        margin: Margen para los pares negativos (float).

    Returns:
        loss: El valor promedio de la pérdida contrastiva en el batch.
    """
    # 1. Calcular la distancia Coseno entre los pares de embeddings
    # Similitud Coseno varía de -1 (opuestos) a 1 (idénticos)
    cos_sim = F.cosine_similarity(embedding1, embedding2)
    # Distancia Coseno varía de 0 (idénticos) a 2 (opuestos)
    distance = 1 - cos_sim

    # 2. Separar por etiquetas
    is_positive = (label == 1)
    is_negative = (label == 0)

    # 3. Calcular pérdidas parciales
    # Para pares POSITIVOS (label==1): La pérdida es la distancia al cuadrado.
    # Queremos minimizar esta distancia (idealmente a 0).
    loss_positive = distance[is_positive].pow(2)

    # Para pares NEGATIVOS (label==0): La pérdida es max(0, margin - distancia)^2.
    # Queremos maximizar la distancia, pero solo hasta el margen.
    loss_negative = F.relu(margin - distance[is_negative]).pow(2) # F.relu(x) es max(0, x)

    # 4. Combinar las pérdidas (promedio sobre el batch)
    # Creamos un tensor para almacenar las pérdidas individuales de cada par
    losses = torch.zeros_like(distance)

    # Asignamos las pérdidas calculadas a sus lugares correspondientes
    # Comprobamos si hay positivos/negativos en el batch para evitar errores en indexación vacía
    if torch.any(is_positive):
        losses[is_positive] = loss_positive
    if torch.any(is_negative):
        losses[is_negative] = loss_negative

    # La pérdida total es la media de las pérdidas individuales del batch
    total_loss = losses.mean()

    return total_loss

# Definimos un margen (este es un hiperparámetro que puedes ajustar)
margin_value = 0.5
print(f"Función de pérdida contrastiva manual definida con margen={margin_value}.")

# --- Notas importantes ---
# * Normalización: Es crucial que los 'embedding1' y 'embedding2' que le pases
#   a esta función ya estén normalizados (norma L2 = 1). Haremos esto en el bucle
#   de entrenamiento ANTES de llamar a esta función.
# * Margen: El valor del 'margin' (ej. 0.5) es un hiperparámetro. Define qué tan
#   separados deben estar los negativos. Puede que necesites experimentar con él.

Función de pérdida contrastiva manual definida con margen=0.5.


In [ ]:
import torch

# Configurar el optimizador
learning_rate = 2e-6 # Ejemplo: 0.000005. ¡Empieza bajo!
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

print(f"Optimizador AdamW configurado con learning rate = {learning_rate}")

Optimizador AdamW configurado con learning rate = 2e-06


Evaluación

In [ ]:
import torch
import torch.nn.functional as F
from tqdm.auto import tqdm

@torch.no_grad() # Importante: Desactiva el cálculo de gradientes para evaluación
def evaluate_retrieval(model_to_eval, dataloader_val, device, top_k_values=(1, 5, 10)):
    """
    Evalúa el modelo en una tarea de retrieval texto-a-imagen usando Recall@k.
    Asume que cada par (texto, imagen) en el dataloader_val es un par correcto.
    """
    model_to_eval.eval() # Poner el modelo en modo evaluación (desactiva dropout, etc.)

    all_image_embeddings_list = []
    all_text_embeddings_list = []

    print("Generando embeddings para el conjunto de validación...")
    for batch in tqdm(dataloader_val, desc="Evaluación - Embeddings"):
        pixel_values = torch.Tensor(np.array(batch['pixel_values']).transpose(3,0,1,2)).to(device)
        input_ids = torch.IntTensor(np.array(batch['input_ids'], dtype=np.int32).transpose(1,0)).to(device)
        attention_mask = torch.Tensor(np.array(batch['attention_mask']).transpose(1,0)).to(device)
        labels = torch.tensor(np.array(batch['labels'])).to(device) # Las etiquetas 0 o 1

        # Solo quedarnos con los indices de labels = 1
        indices_to_keep = torch.where(labels == 1)[0]
        pixel_values = pixel_values[indices_to_keep]
        input_ids = input_ids[indices_to_keep]
        attention_mask = attention_mask[indices_to_keep]

        image_emb = model_to_eval.get_image_features(pixel_values=pixel_values)
        text_emb = model_to_eval.get_text_features(input_ids=input_ids, attention_mask=attention_mask)

        all_image_embeddings_list.append(F.normalize(image_emb, p=2, dim=-1))
        all_text_embeddings_list.append(F.normalize(text_emb, p=2, dim=-1))

    if not all_image_embeddings_list or not all_text_embeddings_list:
        print("Advertencia: No se generaron embeddings de validación. ¿Dataloader vacío?")
        return {f"R@{k}": 0.0 for k in top_k_values}

    # Concatenar todos los embeddings de los batches
    gallery_image_embeddings = torch.cat(all_image_embeddings_list, dim=0)
    query_text_embeddings = torch.cat(all_text_embeddings_list, dim=0)

    num_queries = query_text_embeddings.shape[0]
    if num_queries == 0:
        print("Advertencia: Cero consultas de texto para evaluar.")
        return {f"R@{k}": 0.0 for k in top_k_values}
    if gallery_image_embeddings.shape[0] != num_queries:
        print(f"Advertencia: Desajuste en el número de embeddings de imagen ({gallery_image_embeddings.shape[0]}) y texto ({num_queries}).")
        # Esto no debería pasar si el dataloader de validación tiene pares consistentes.
        # Para este tipo de Recall@k (donde el target i-ésimo es la imagen i-ésima), necesitamos que coincidan.
        # Podrías optar por tomar el mínimo de ambos si es necesario, o investigar el error.
        # Por simplicidad, asumiremos que son iguales o retornaremos 0.
        min_len = min(gallery_image_embeddings.shape[0], num_queries)
        gallery_image_embeddings = gallery_image_embeddings[:min_len]
        query_text_embeddings = query_text_embeddings[:min_len]
        num_queries = min_len
        if num_queries == 0: return {f"R@{k}": 0.0 for k in top_k_values}


    print(f"Calculando similitudes para {num_queries} consultas...")
    # Calcular similitudes texto-a-imagen: (N_queries, E) @ (E, N_gallery) -> (N_queries, N_gallery)
    # Aquí, N_queries == N_gallery == num_queries
    similarity_matrix = torch.matmul(query_text_embeddings, gallery_image_embeddings.t())

    # Para cada consulta de texto i-ésima, su imagen correcta es la i-ésima imagen en la galería
    # (porque el dataloader de validación mantiene el orden de los pares correctos)
    ground_truth_indices = torch.arange(num_queries, device=device)

    recall_scores = {}
    for k in top_k_values:
        # Obtener los índices de las k imágenes más similares para cada consulta
        # similarity_matrix: [num_queries, num_gallery_images]
        # top_k_indices: [num_queries, k]
        # Asegurarse de que k no sea mayor que el número de imágenes en la galería
        actual_k = min(k, gallery_image_embeddings.shape[0])
        if actual_k == 0: # No hay imágenes en la galería
            recall_scores[f"R@{k}"] = 0.0
            continue

        _, top_k_indices = torch.topk(similarity_matrix, k=actual_k, dim=1)

        # Comparar si el índice de la imagen correcta está en los top_k recuperados
        # ground_truth_indices.view(-1, 1) hace que sea [num_queries, 1] para broadcasting
        # correct_hits_in_top_k es un tensor booleano [num_queries]
        correct_hits_in_top_k = (ground_truth_indices.view(-1, 1) == top_k_indices).any(dim=1)

        # Recall@k es el promedio de aciertos
        recall_at_k = correct_hits_in_top_k.float().mean().item()
        recall_scores[f"R@{k}"] = recall_at_k

    model_to_eval.train() # Volver a poner el modelo en modo entrenamiento después de la evaluación
    return recall_scores

print("Función de evaluación de retrieval (evaluate_retrieval) definida.")

Función de evaluación de retrieval (evaluate_retrieval) definida.


In [ ]:
import torch.nn.functional as F
import numpy as np
from tqdm.auto import tqdm # Para mostrar una barra de progreso bonita

# --- Hiperparámetros de Entrenamiento ---
num_epochs = 3     # Número de épocas. Empieza con pocas (ej. 3-5) para probar.
margin = 0.5       # Margen para nuestra manual_contrastive_loss

# --- Preparación del Dispositivo ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device) # Mueve el modelo a la GPU si existe, si no a la CPU
print(f"Entrenando en el dispositivo: {device}")

print("\nIniciando Evaluación de Validación...")
# Asegúrate de que 'processed_dataset_validation' y 'val_loader' estén definidos
if 'val_loader' in locals() or 'val_loader' in globals():
    metrics = evaluate_retrieval(model, val_loader, device, top_k_values=(1, 5, 10, 15, 20))
    for metric_name, score in metrics.items():
        print(f"Validación {metric_name}: {score:.4f}")
else:
    print("DataLoader de validación no definido. Saltando evaluación.")

# --- Bucle Principal de Entrenamiento ---
for epoch in range(num_epochs):
    print(f"\n--- Iniciando Época {epoch+1}/{num_epochs} ---")

    model.train() # MUY IMPORTANTE: Poner el modelo en modo entrenamiento

    total_loss_epoch = 0.0 # Para calcular la pérdida promedio de la época

    progress_bar = tqdm(train_loader, desc=f"Época {epoch+1}")

    for batch_idx, batch in enumerate(progress_bar):
        # 1. Mover datos del batch al dispositivo correcto
        pixel_values = torch.Tensor(np.array(batch['pixel_values']).transpose(3,0,1,2)).to(device)
        input_ids = torch.IntTensor(np.array(batch['input_ids'], dtype=np.int32).transpose(1,0)).to(device)
        attention_mask = torch.Tensor(np.array(batch['attention_mask']).transpose(1,0)).to(device)
        labels = torch.tensor(np.array(batch['labels'])).to(device) # Las etiquetas 0 o 1

        # 2. Poner a cero los gradientes acumulados del paso anterior
        optimizer.zero_grad()

        # 3. Forward Pass: obtener embeddings
        image_embeddings = model.get_image_features(pixel_values=pixel_values)
        text_embeddings = model.get_text_features(input_ids=input_ids, attention_mask=attention_mask)

        # 4. Normalizar los Embeddings (Norma L2)
        image_embeddings_norm = F.normalize(image_embeddings, p=2, dim=-1)
        text_embeddings_norm = F.normalize(text_embeddings, p=2, dim=-1)

        # 5. Calcular la Pérdida
        # Usamos nuestra función manual_contrastive_loss
        loss = manual_contrastive_loss(
            image_embeddings_norm,
            text_embeddings_norm,
            labels,
            margin=margin
        )

        # Manejar posible NaN en la pérdida (puede ocurrir si algo va mal)
        if torch.isnan(loss):
            print(f"¡PÉRDIDA NaN DETECTADA en epoch {epoch+1}, batch {batch_idx}! Saltando este batch.")
            # Podrías querer detener el entrenamiento o investigar más a fondo.
            # Por ahora, simplemente nos saltamos el paso del optimizador para este batch.
            optimizer.zero_grad() # Asegúrate de limpiar gradientes igual
            continue # Salta al siguiente batch

        # 6. Backward Pass: Calcular los gradientes
        loss.backward()

        # 7. Paso del Optimizador: Actualizar los pesos
        optimizer.step()

        # 8. (Opcional) Registrar pérdida y actualizar barra de progreso
        loss_item = loss.item()
        total_loss_epoch += loss_item
        progress_bar.set_postfix({'loss': f'{loss_item:.4f}'})

    # --- Fin de la Época ---
    avg_epoch_loss = total_loss_epoch / len(train_loader)
    print(f"Fin de Época {epoch+1}. Pérdida promedio: {avg_epoch_loss:.4f}")

    print("\nIniciando Evaluación de Validación...")
    # Asegúrate de que 'processed_dataset_validation' y 'val_loader' estén definidos
    if 'val_loader' in locals() or 'val_loader' in globals():
        metrics = evaluate_retrieval(model, val_loader, device, top_k_values=(1, 5, 10, 15, 20))
        for metric_name, score in metrics.items():
            print(f"Validación {metric_name}: {score:.4f}")
    else:
        print("DataLoader de validación no definido. Saltando evaluación.")

    # --- Guardado del Modelo (Checkpointing - MUY RECOMENDADO) ---
    output_dir = f"./clip_finetuned_epoch_{epoch+1}"
    model.save_pretrained(output_dir)
    processor.save_pretrained(output_dir)
    print(f"Modelo guardado en: {output_dir}")

print("\n--- Entrenamiento Completado ---")

# Puedes guardar el modelo final aquí también
final_output_dir = "./clip_finetuned_final"
model.save_pretrained(final_output_dir)
processor.save_pretrained(final_output_dir)

Entrenando en el dispositivo: cuda

Iniciando Evaluación de Validación...
Generando embeddings para el conjunto de validación...


Evaluación - Embeddings:   0%|          | 0/9 [00:00<?, ?it/s]

Calculando similitudes para 372 consultas...
Validación R@1: 0.0000
Validación R@5: 0.0054
Validación R@10: 0.0215
Validación R@15: 0.0349
Validación R@20: 0.0538

--- Iniciando Época 1/3 ---


Época 1:   0%|          | 0/118 [00:00<?, ?it/s]

Fin de Época 1. Pérdida promedio: 0.1571

Iniciando Evaluación de Validación...
Generando embeddings para el conjunto de validación...


...

Época 3:   0%|          | 0/118 [00:00<?, ?it/s]

Fin de Época 3. Pérdida promedio: 0.0583

Iniciando Evaluación de Validación...
Generando embeddings para el conjunto de validación...


Evaluación - Embeddings:   0%|          | 0/9 [00:00<?, ?it/s]

Calculando similitudes para 372 consultas...
Validación R@1: 0.0027
Validación R@5: 0.0161
Validación R@10: 0.0269
Validación R@15: 0.0511
Validación R@20: 0.0672
Modelo guardado en: ./clip_finetuned_epoch_3

--- Entrenamiento Completado ---


[]

FIN

In [ ]:
from transformers import TrainingArguments
from sentence_transformers import SentenceTransformer, SentenceTransformerTrainer, losses, SentenceTransformerTrainingArguments
from transformers import Trainer

model = SentenceTransformer("sentence-transformers/clip-ViT-L-14")

# pick specific layers to train (note: you can add more layers to this list)
trainable_layers_list = ['projection']

# Apply freezing configuration
for name, param in model.named_parameters():

    # freeze all params
    param.requires_grad = False

    # unfreeze layers in trainable_layers_list
    if any(layer in name for layer in trainable_layers_list):
        param.requires_grad = True

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")
model.to(device)

loss = losses.ContrastiveLoss(model)

training_args = SentenceTransformerTrainingArguments("arguments-fine-tuning", disable_tqdm=False,)

# trainer = Trainer(
#     model,
#     training_args,
#     train_dataset=processed_dataset["train"],
#     eval_dataset=processed_dataset["valid"],
#     tokenizer=processor,
#     compute_metrics=None,
#     loss_fn=loss,
#     # processing_class=tokenizer,
# )


trainer = SentenceTransformerTrainer(
    model=model,
    train_dataset=processed_dataset["train"],
    eval_dataset=processed_dataset["valid"],
    loss=loss,
    args=training_args,
    tokenizer=processor,
    compute_metrics=None,
)
trainer.train()

modules.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/118 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/1.91k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/733 [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/862k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.71G [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/354 [00:00<?, ?B/s]

In [ ]:
!pip install wandb


In [ ]:
import torch
import torch.nn.functional as F
from tqdm.auto import tqdm
import numpy as np
import wandb # <--- Importar W&B

# (Asumimos model, processor, tokenizer, train_dataloader, manual_contrastive_loss, optimizer están definidos)

# --- Hiperparámetros y Configuración ---
 # El que estés usando
num_epochs = 5     # Ejemplo, podrías querer aumentarlo
margin = 0.5       # Margen para la pérdida

# --- 1. Inicializar Weights & Biases ---
# Esto crea un nuevo "run" (ejecución) en W&B
wandb.init(
    project="clip-finetuning-sangre-periferica", # Nombre de tu proyecto en W&B
    config={ # Configura los hiperparámetros que quieres registrar
        "learning_rate": learning_rate,
        "epochs": num_epochs,
        "batch_size": 16,
        "margin": margin,
        "model_name": model, # 'openai/clip-vit-base-patch32'
        "loss_type": "manual_contrastive_loss"
    }
)

# (Optativo) W&B puede observar tu modelo para registrar gradientes y topología (puede ralentizar un poco)
wandb.watch(model, log="all", log_freq=100) # log_freq es cada cuántos batches

# --- Preparación del Dispositivo ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=wandb.config.learning_rate) # Usar config de W&B

print(f"Entrenando en el dispositivo: {device}")
print(f"Configuración para W&B: {wandb.config}")


# --- Bucle Principal de Entrenamiento ---
for epoch in range(wandb.config.epochs): # Usar el número de épocas de la config de W&B
    print(f"\n--- Iniciando Época {epoch+1}/{wandb.config.epochs} ---")
    model.train()
    total_loss_epoch = 0.0
    progress_bar = tqdm(train_loader, desc=f"Época {epoch+1}")

    for batch_idx, batch in enumerate(progress_bar):
        pixel_values = torch.Tensor(np.array(batch['pixel_values']).transpose(3,0,1,2)).to(device)
        input_ids = torch.IntTensor(np.array(batch['input_ids'], dtype=np.int32).transpose(1,0)).to(device)
        attention_mask = torch.Tensor(np.array(batch['attention_mask']).transpose(1,0)).to(device)
        labels = torch.tensor(np.array(batch['labels'])).to(device)

        optimizer.zero_grad()

        image_embeddings = model.get_image_features(pixel_values=pixel_values)
        text_embeddings = model.get_text_features(input_ids=input_ids, attention_mask=attention_mask)

        image_embeddings_norm = F.normalize(image_embeddings, p=2, dim=-1)
        text_embeddings_norm = F.normalize(text_embeddings, p=2, dim=-1)

        loss = manual_contrastive_loss(
            image_embeddings_norm,
            text_embeddings_norm,
            labels,
            margin=wandb.config.margin # Usar el margen de la config de W&B
        )

        if torch.isnan(loss):
            print(f"¡PÉRDIDA NaN DETECTADA en epoch {epoch+1}, batch {batch_idx}! Saltando este batch.")
            optimizer.zero_grad()
            continue

        loss.backward()
        optimizer.step()

        loss_item = loss.item()
        total_loss_epoch += loss_item
        progress_bar.set_postfix({'loss': f'{loss_item:.4f}'})

        # --- 2. Registrar la pérdida del batch en W&B ---
        # El 'step' puede ser el número global de batches procesados
        global_step = epoch * len(train_loader) + batch_idx
        wandb.log({"batch_train_loss": loss_item, "global_step": global_step, "epoch": epoch +1})

    avg_epoch_loss = total_loss_epoch / len(train_loader)
    print(f"Fin de Época {epoch+1}. Pérdida promedio de entrenamiento: {avg_epoch_loss:.4f}")

    # --- 3. Registrar la pérdida promedio de la época en W&B ---
    wandb.log({"avg_epoch_train_loss": avg_epoch_loss, "epoch": epoch + 1})

    # --- Evaluación (Integrar con W&B) ---
    print("\nIniciando Evaluación de Validación...")
    if 'val_loader' in locals() or 'val_loader' in globals():
        metrics = evaluate_retrieval(model, val_loader, device, top_k_values=(1, 5, 10, 15, 20))
    # --- 4. Registrar métricas de validación en W&B ---
        log_val_metrics = {f"val_{metric_name}": score for metric_name, score in metrics.items()}
        log_val_metrics["epoch"] = epoch + 1 # Asegurar que la época se registre
        wandb.log(log_val_metrics)
        for metric_name, score in metrics.items():
           print(f"Validación {metric_name}: {score:.4f}")
        else:
           print("DataLoader de validación no definido. Saltando evaluación.")

    # (Guardado del Modelo - Opcional en cada época)

print("\n--- Entrenamiento Completado ---")
# --- 5. Finalizar el run de W&B ---
wandb.finish()

batch_train_loss,▆██▄▆▆▅▃▆▄▄▅▅▃▃▂▅▄▄▅▄▃▂▄▂▂▃▄▄▂▃▂▅▃▄▂▄▁▂▂
epoch,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
global_step,▁▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇███
batch_train_loss,0.22878
epoch,1
global_step,62


Entrenando en el dispositivo: cuda
Configuración para W&B: {'learning_rate': 2e-06, 'epochs': 5, 'batch_size': 16, 'margin': 0.5, 'model_name': 'CLIPModel(\n  (text_model): CLIPTextTransformer(\n    (embeddings): CLIPTextEmbeddings(\n      (token_embedding): Embedding(49408, 512)\n      (position_embedding): Embedding(77, 512)\n    )\n    (encoder): CLIPEncoder(\n      (layers): ModuleList(\n        (0-11): 12 x CLIPEncoderLayer(\n          (self_attn): CLIPSdpaAttention(\n            (k_proj): Linear(in_features=512, out_features=512, bias=True)\n            (v_proj): Linear(in_features=512, out_features=512, bias=True)\n            (q_proj): Linear(in_features=512, out_features=512, bias=True)\n            (out_proj): Linear(in_features=512, out_features=512, bias=True)\n          )\n          (layer_norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)\n          (mlp): CLIPMLP(\n            (activation_fn): QuickGELUActivation()\n            (fc1): Linear(in_features=512, o

Época 1:   0%|          | 0/118 [00:00<?, ?it/s]

Fin de Época 1. Pérdida promedio de entrenamiento: 0.1768

Iniciando Evaluación de Validación...
Generando embeddings para el conjunto de validación...


Evaluación - Embeddings:   0%|          | 0/9 [00:00<?, ?it/s]

Calculando similitudes para 372 consultas...
Validación R@1: 0.0027
Validación R@5: 0.0188
Validación R@10: 0.0296
Validación R@15: 0.0430
Validación R@20: 0.0618
DataLoader de validación no definido. Saltando evaluación.

--- Iniciando Época 2/5 ---


Época 2:   0%|          | 0/118 [00:00<?, ?it/s]

With augmentation

In [ ]:
import torch
import torchvision.transforms as T
from PIL import Image

# Definir la transformación de rotación aleatoria
# 'degrees' especifica el rango de rotación: (-grados, +grados).  Ej: degrees=10 rota entre -10 y +10 grados
# Por simplicidad, usaré un rango de rotación de hasta 10 grados.  Puedes ajustarlo.
# Si quieres rotaciones de 0 a 360, usa degrees=(0, 360), pero ten cuidado con la semántica de tu imagen.
random_rotation = T.RandomRotation(degrees=10)

# Para usarlo dentro de tu función preprocess_data:

def preprocess_data_with_augmentation(examples):
    images = [Image.open(path).convert("RGB") for path in examples['image_path']]

    # Aplicar la rotación aleatoria a CADA imagen.  Es importante hacerlo aquí.
    augmented_images = [random_rotation(img) for img in images]

    inputs = processor(
        text=examples['caption'],
        images=augmented_images, # Usamos las imágenes aumentadas
        truncation=True,
        padding=False
    )
    inputs['label'] = examples['similarity']
    return inputs

print("Transformación de rotación aleatoria definida. Ejemplo de uso:")
print("""
def preprocess_data_with_augmentation(examples):
    # ... (código para cargar las imágenes) ...
    augmented_images = [random_rotation(img) for img in images]
    # ... (resto del preprocesamiento) ...
""")

In [ ]:
processed_dataset_train = dataset_dict['train'].map(
    preprocess_data_with_augmentation,  # ¡Usa la versión aumentada!
    batched=True,
    remove_columns=dataset_dict['train'].column_names
)